In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl
from tensorflow.keras.initializers import VarianceScaling

class ProbabilityDistribution(tf.keras.Model):
    def call(self, logits, **kwargs):
        # Random distribution
        return tf.squeeze(tf.random.categorical(logits, 1), axis=-1)
    
class Model(tf.keras.Model):
    def __init__(self, num_actions, hidden):
        # Note: no tf.get_variable(), just simple Keras API!
        super().__init__('mlp_policy')
        self.normalize = kl.Lambda(lambda layer: layer / 255)    # normalize by 255
        self.conv1 = kl.Conv2D(32, (8, 8), strides=4, kernel_initializer=VarianceScaling(scale=2.), activation='relu', use_bias=False)
        self.conv2 = kl.Conv2D(64, (4, 4), strides=2, kernel_initializer=VarianceScaling(scale=2.), activation='relu', use_bias=False)
        self.conv3 = kl.Conv2D(64, (3, 3), strides=1, kernel_initializer=VarianceScaling(scale=2.), activation='relu', use_bias=False)
        self.conv4 = kl.Conv2D(hidden, (7, 7), strides=1, kernel_initializer=VarianceScaling(scale=2.), activation='relu', use_bias=False)
        
        self.flatten = kl.Flatten()
        self.value = kl.Dense(1, kernel_initializer=VarianceScaling(scale=2.), name="value")
        self.logits = kl.Dense(num_actions, kernel_initializer=VarianceScaling(scale=2.), name='policy_logits')
        
        self.dist = ProbabilityDistribution()
    def call(self, inputs, **kwargs):
        # Inputs is a numpy array, convert to a tensor.
        x = tf.convert_to_tensor(inputs)
        # Separate hidden layers from the same input tensor.
        x = self.normalize(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        return self.logits(x), self.value(x)

In [ ]:
class Agent:
    def __init__(self, model, save_path=PATH_SAVE_MODEL, load_path=PATH_LOAD_MODEL, lr=LR, gamma=GAMMA, value_c=VALUE_C,
                 entropy_c=ENTROPY_C, clip_ratio=CLIP_RATIO, std_adv=STD_ADV, agent=AGENT, input_shape=INPUT_SHAPE,
                 batch_size = BATCH_SIZE, updates=N_UPDATES):
        # Coefficients are used for the loss terms.
        self.value_c = value_c
        self.entropy_c = entropy_c
        # `gamma` is the discount factor
        self.gamma = gamma
        self.save_path = save_path
        self.load_path = load_path
        self.clip_ratio = clip_ratio
        self.std_adv = std_adv
        self.agent = agent
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.updates = updates
        self.opt = opt.RMSprop(lr=lr)
        
        self.model = model
        if load_path is not None:
            print("loading model in {}".format(load_path))
            self.load_model(load_path)
            print("model loaded")
            
            
    def train(self, wrapper):
        # Storage helpers for a single batch of data.
        actions = np.empty((self.batch_size), dtype=np.int32)
        rewards, dones, values = np.empty((3, self.batch_size))
        observations = np.empty((self.batch_size,) + self.input_shape)
        old_logits = np.empty((self.batch_size, wrapper.env.action_space.n), dtype=np.float32)
        # Training loop: collect samples, send to optimizer, repeat updates times.
        ep_rewards = [0.0]
        next_obs = wrapper.reset()
        for update in tqdm(range(self.updates)):
            start_time = time.time()
            for step in range(self.batch_size):
                observations[step] = next_obs.copy()
                old_logits[step], actions[step], values[step] = self.logits_action_value(next_obs[None, :])
                next_obs, rewards[step], dones[step] = wrapper.step(actions[step])
                next_obs = wrapper.state
                ep_rewards[-1] += rewards[step]
                if dones[step]:
                    ep_rewards.append(0.0)
                    next_obs = wrapper.reset()
                    wandb.log({'Game number': len(ep_rewards) - 1, '# Update': update, '% Update': round(update / self.updates, 2),
                                "Reward": round(ep_rewards[-2], 2), "Time taken": round(time.time() - start_time, 2)})
            _, _, next_value = self.logits_action_value(next_obs[None, :])
            returns, advs = self._returns_advantages(rewards, dones, values, next_value, self.std_adv)
            # Performs a full training step on the collected batch.
            # Note: no need to mess around with gradients, Keras API handles it.
            with tf.GradientTape() as tape:
                logits, v = self.model(observations, training=True)
                if self.agent == "A2C":
                    logit_loss = self._logits_loss_a2c(actions, advs, logits)
                elif self.agent == "PPO":
                    logit_loss = self._logits_loss_ppo(old_logits, logits, actions, advs, wrapper.env.action_space.n)
                else:
                    raise Exception("Sorry agent can be just A2C or PPO")
                value_loss = self._value_loss(returns, v)
                loss = logit_loss + value_loss
            grads = tape.gradient(loss, self.model.trainable_variables)
            self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
            if update % 5000 == 0 and self.save_path is not None:
                print("Saving model in {}".format(self.save_path))
                self.save_model(f'{self.save_path}/save_agent_{time.strftime("%Y%m%d%H%M") + "_" + str(update).zfill(8)}/model.tf')
                print("model saved")
                
        return ep_rewards

In [ ]:
def _returns_advantages(self, rewards, dones, values, next_value, standardize_adv):
        returns = np.append(np.zeros_like(rewards), next_value, axis=-1)
        # Returns are calculated as discounted sum of future rewards.
        for t in reversed(range(rewards.shape[0])):
            returns[t] = rewards[t] + self.gamma * returns[t + 1] * (1 - dones[t])
        returns = returns[:-1]
        # Advantages are equal to returns - baseline (value estimates in our case).
        advantages = returns - values
        
        if standardize_adv:
            advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-10)
        return returns, advantages

In [ ]:
def _value_loss(self, returns, value):
        # Value loss is typically MSE between value estimates and returns.
        return self.value_c * kloss.mean_squared_error(returns, value)

In [ ]:
def _logits_loss_a2c(self, actions, advantages, logits):
        # Sparse categorical CE loss obj that supports sample_weight arg on `call()`.
        # `from_logits` argument ensures transformation into normalized probabilities.
        weighted_sparse_ce = kloss.SparseCategoricalCrossentropy(from_logits=True)
        # Policy loss is defined by policy gradients, weighted by advantages.
        # Note: we only calculate the loss on the actions we've actually taken.
        actions = tf.cast(actions, tf.int32)
        policy_loss = weighted_sparse_ce(actions, logits, sample_weight=advantages)
        # Entropy loss can be calculated as cross-entropy over itself.
        probs = tf.nn.softmax(logits)
        entropy_loss = kloss.categorical_crossentropy(probs, probs)
        # We want to minimize policy and maximize entropy losses.
        # Here signs are flipped because the optimizer minimizes.
        return policy_loss - self.entropy_c * entropy_loss

In [ ]:
wandb.init(
  project="tensorflow2_pong_{}".format(AGENT.lower()),
  tags=[AGENT.lower(), "CNN", "RL", "atari_pong"],
  config=CONFIG_WANDB,
)

In [ ]:
pw = PongWrapper(ENV_NAME, history_length=4)
model = Model(num_actions=pw.env.action_space.n, hidden=HIDDEN)
agent = Agent(model)

In [ ]:
def main():
    rewards_history = agent.train(pw)
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        # Save the model, I need this in order to save the networks, frame number, rewards and losses. 
        # if I want to stop the script and restart without training from the beginning
        if PATH_SAVE_MODEL is None:
            print("Setting path to ../model/{}".format(AGENT.lower()))
            PATH_SAVE_MODEL = "../model/{}".format(AGENT.lower())
        print('Saving the model in ' + f'{PATH_SAVE_MODEL}/save_agent_{time.strftime("%Y%m%d%H%M")}')
        agent.save_model(f'{PATH_SAVE_MODEL}/save_agent_{time.strftime("%Y%m%d%H%M")}/model.tf')
        print('Saved.')